### Ejercicio 7 Penguins

#### Pablo Carmona Artal

In [33]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline

In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

penguins_data = pd.read_csv("/workspaces/Modelos_IA/penguins.csv")
#Con esto vemos si hubiese algun NaN que tuviesemos que tratar 
penguins_data.isna().sum()
#Vemos que hay entonces propongo eliminarlos ya que no nos van a proporcionar informacion
penguins_data = penguins_data.dropna(subset='sex')
penguins_data.isna().sum()



species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
year                 0
dtype: int64

In [35]:
penguins_data

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


In [36]:
#Dividimos los datos en X e Y del entrenamiento
Y_penguins = penguins_data['species'] #target
X_penguins = penguins_data[penguins_data.columns[1:].values]


In [37]:
#Tres especies de nuesto ouput
set(Y_penguins)

{'Adelie', 'Chinstrap', 'Gentoo'}

In [38]:
#Dividimos el train y validacion en 80% y 20% respectivamente
X_train, X_test, y_train, y_test = train_test_split(X_penguins, Y_penguins, test_size=70, random_state=123,
                                                                        shuffle=True, stratify=Y_penguins)
print(X_train.shape)
print(X_test.shape)

(263, 7)
(70, 7)


In [39]:
#Estandarizamos los datos para manatener los datos y el error regularizado 
penguins_data.dtypes
#El problema es que hay datos no numericos, los cuales no se pueden normalizar, y tampoco analizar por un modelo de redes neuronales.
#Por ello dividimos los datos numericos para poder estandarizarlos
num_values = X_train.select_dtypes(include="float64").columns
scaler = StandardScaler()
scaler.fit(X_train[num_values].values)
X_train[num_values] = scaler.transform(X_train[num_values].values)
X_test[num_values] = scaler.transform(X_test[num_values].values)

#A continuacion hacemos la codificacion de los valores categoricos 
from sklearn.preprocessing import LabelEncoder
#Creamos un label encoder para cada uno
#Para Island
island_label= LabelEncoder()
island_label.fit(X_train['island'].values)
#Comprobamos que la transfromacion sea adecuada
print(island_label.classes_)
print(island_label.transform(island_label.classes_))
#y transformamos
X_train['island'] = island_label.transform(X_train['island'].values)
X_test['island'] = island_label.transform(X_test['island'].values)
#Para el genero
Gender_label = LabelEncoder()
Gender_label.fit(X_train['sex'].values)
print(Gender_label.classes_)
print(Gender_label.transform(Gender_label.classes_))
X_train['sex'] = Gender_label.transform(X_train['sex'].values)
X_test['sex'] = Gender_label.transform(X_test['sex'].values)


#Para el target (species)
target_label = LabelEncoder()
target_label.fit(y_train)
print(target_label.classes_)
print(target_label.transform(target_label.classes_))
y_train = target_label.transform(y_train)
y_test = target_label.transform(y_test)

['Biscoe' 'Dream' 'Torgersen']
[0 1 2]
['female' 'male']
[0 1]
['Adelie' 'Chinstrap' 'Gentoo']
[0 1 2]


In [40]:
X_train

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
98,1,-1.975967,-0.544048,-1.597313,-1.641909,0,2008
341,1,1.006126,0.517336,-0.545678,-0.547105,1,2009
165,0,0.789246,-1.302180,0.856503,2.049145,1,2007
278,1,1.313372,1.022757,-0.545678,-0.703505,1,2007
272,0,0.500074,-1.453806,0.996721,0.797940,0,2009
...,...,...,...,...,...,...,...
91,1,-0.530104,0.466794,0.295631,0.109778,1,2008
276,1,0.445854,0.365710,-0.615787,-0.891186,0,2007
157,0,0.445854,-1.858143,0.646176,0.422579,0,2007
151,1,-0.457811,0.668962,0.015195,-0.265584,1,2009


In [41]:
#Comporbamos que los datos se hayan normalizado adecuadamente
print("Entrenamiento (media no std): {}".format(np.mean(X_train, axis=0)))
print("Entrenamiento (media no std): {}".format(np.mean(X_test, axis=0)))
print("Entrenamiento: (desv. tipica std) {}".format(np.std(X_train, axis=0)))
print("Entrenamiento: (desv. tipica std) {}".format(np.std(X_test, axis=0)))
#vemos como efectivamente las medias de los valores numericos tranformados estan en torno a 0 y las desviaciones estandar en torno a 1.



Entrenamiento (media no std): island               6.501901e-01
bill_length_mm       1.160880e-16
bill_depth_mm       -8.105050e-17
flipper_length_mm   -7.564714e-16
body_mass_g         -4.930572e-16
sex                  5.019011e-01
year                 2.008019e+03
dtype: float64
Entrenamiento (media no std): island                  0.657143
bill_length_mm         -0.034638
bill_depth_mm          -0.027797
flipper_length_mm       0.061266
body_mass_g            -0.030983
sex                     0.514286
year                 2008.128571
dtype: float64
Entrenamiento: (desv. tipica std) island               0.713310
bill_length_mm       1.000000
bill_depth_mm        1.000000
flipper_length_mm    1.000000
body_mass_g          1.000000
sex                  0.499996
year                 0.815499
dtype: float64
Entrenamiento: (desv. tipica std) island               0.714857
bill_length_mm       0.935454
bill_depth_mm        0.969825
flipper_length_mm    0.905224
body_mass_g          1.02774

In [42]:
#Nos aseguramos de coger tan solo los values
X_train = X_train.values
X_test = X_test.values


In [43]:
#Usamos dos variables explicativas como recomienda el enunciado
tX_train = torch.tensor(X_train, requires_grad=False, dtype=torch.float)[:, 0:2]
tX_test = torch.tensor(X_test, requires_grad=False, dtype=torch.float)[:, 0:2]
ty_train = torch.tensor(y_train, requires_grad=False, dtype=torch.long)  
ty_test = torch.tensor(y_test, requires_grad=False, dtype=torch.long) 

In [44]:
#Observamos como las medidad concuerdan
print(tX_train.shape)
print(tX_test.shape)
print(ty_train.shape)
print(ty_test.shape)

torch.Size([263, 2])
torch.Size([70, 2])
torch.Size([263])
torch.Size([70])


A continuacion comenzaremos con la construccion de la red neuronal. En nuesto caso se compondra de una red de 7 neuronas de entrada y tres de salida, la red nueronal nos dara la probabilidad de que se pertenezca a cualquiera de las tres especies {'Adelie', 'Chinstrap', 'Gentoo'}.

In [45]:
#Construimos nuestra red neuronal con 80 neuronas en nuestra capa intermedia y como funcion de activacion de esta capa usaremos la funcion Relu

#Con la funcion Sequential somos capaces de encadenar procesos, y pasan al siguiente modulo a traves de fowards
nn1 = torch.nn.Sequential(
    torch.nn.Linear(tX_train.shape[1], 80), # capa lineal con 7 entradas y 80 salidas
    torch.nn.ReLU(),  # activación ReLU
    torch.nn.Linear(80, len(target_label.classes_)), # capa lineal con 80 entradas y 3 salidas. 
    torch.nn.LogSoftmax(dim=1) # El output es la probabilidad de pertenencia a cada categoría (class)
)
#A traves dela funcion Softmax (o LogSoftMax), nos permiten saber las probabilidades de que pertenezcan a una variable categorica especifica.
#Usamos LogSoftMax porque devuelve unas probabilidades mas estables que el original. 

In [46]:
#Comprobacion por NaNs
tiene_nan = torch.isnan(tX_train).any().item()
if tiene_nan:
    print("El tensor tiene al menos un valor NaN.")
else:
    print("El tensor no tiene valores NaN.")

El tensor no tiene valores NaN.


In [47]:
print(torch.exp(nn1(tX_train[0:5,:])))
print(ty_train[0:5])
#Vemos como al usar todas las variables como el modelo nos da probabilidades erroneas, para ello he pensado en reducir el numero de variables que incluimos en el analisis


tensor([[0.1954, 0.3291, 0.4755],
        [0.3080, 0.3234, 0.3686],
        [0.3516, 0.2621, 0.3863],
        [0.3200, 0.3137, 0.3663],
        [0.3463, 0.2705, 0.3833]], grad_fn=<ExpBackward0>)
tensor([0, 1, 2, 1, 2])


In [48]:
#Calculamos el valor de perdidas del modelo 
#Para las 50 primeros casos. Ejemplo
torch.nn.functional.nll_loss(nn1(tX_train), ty_train)

tensor(1.2159, grad_fn=<NllLossBackward0>)

In [49]:
#A continuacion creamos las semillas iniciales que genera numeros aleatorios, para que sea punto de partida para la creacion de valores iniciales para nuestros parametros usados en el entrenamiento
#Hay varias formas de inicializar los pesos, en nuestro caso al ser una red nuronal simple los iniciaremos con un numero aleatorio entre 1 y 0 
def do_seeds(sn):
    np.random.seed(sn)
    torch.manual_seed(sn)
    random.seed(sn)
do_seeds(0)

do_seeds(12)
for param in nn1.parameters():
    torch.nn.init.uniform_(param, 0, 1) 

In [50]:
#Una vez inicializados los parametros, podemo iniciar el entrenamiento, donde buscaremos reducir el error, a traves del Gradiente Descendiente. 
#Hay dos tipos el estandard y el escolastico, el cual usaremos por su eficacia y rapidez.

opt = torch.optim.SGD(nn1.parameters(), lr=0.02) #En este caso usaremos un learning rate del 0.02


In [51]:
for i in range(10000):  
    pred_y = nn1(tX_train) # calculamos nuestras predicciones con los parámetros actuales
    loss = torch.nn.functional.nll_loss(pred_y, ty_train) # calculamos la pérdida con los resultados 
    
    if i%1000 == 0:
        print("Época: {}, Pérdida: {:.4f}".format(i, loss.item()))
        loss.backward() # propagamos las derivadas de la 'loss' con respecto a nuestros parámetros
        opt.step() # ejecutamos un paso de nuestro optimizador
        opt.zero_grad() # reseteamos los valores acumulados de las derivadas para que no las acumule

Época: 0, Pérdida: 2.2929
Época: 1000, Pérdida: 1.9003
Época: 2000, Pérdida: 1.5535
Época: 3000, Pérdida: 1.2724
Época: 4000, Pérdida: 1.0865
Época: 5000, Pérdida: 0.9831
Época: 6000, Pérdida: 0.9187
Época: 7000, Pérdida: 0.8695
Época: 8000, Pérdida: 0.8282
Época: 9000, Pérdida: 0.7925


Vemos como ha reducido la perdida bastante gracias al uso del Gradiente Descendente Escolastico 

In [52]:
#Ahora ya tenemos nuestra red entrenada, por lo que podemos observar realizar predicciones con certeza

#Usamos previamente requires_grad to liberar espacio
for param in nn1.parameters():
    param.requires_grad = False

#Continuamos creando una funcion donde demos datos en forma torch y un modelo, y que nos de los valores numericos que tienen la mayor probabilidad (predicciones)
def pred_f(x, forw):
    x_t = torch.tensor(x, dtype=torch.float, requires_grad=False)
    _,y_pred = torch.max(forw(x_t), dim=1)
    return y_pred

#Probamos con las 10 primeras filas
print(pred_f(tX_train[0:10,:],nn1))
print(ty_train[0:10])
#Unicamente falla en el primero, pero por ahora se ve bastante preciso

tensor([0, 1, 2, 1, 2, 0, 0, 0, 2, 0])
tensor([0, 1, 2, 1, 2, 0, 0, 0, 2, 0])


/tmp/ipykernel_2122/2424509203.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_t = torch.tensor(x, dtype=torch.float, requires_grad=False)


In [53]:
#Creamos una funcion de score donde nos de el % de aciertos comparado con la y extraida al principio
def score_f(x, y, forw):
    y_label = torch.tensor(y, dtype=torch.long, requires_grad=False)   # recupera el valor histórico de y
    y_pred = pred_f(x, forw)                                           # obtiene la predicción de y
    
    # devuelve el % aciertos (cuando y coincide con y_pred)
    return torch.sum(y_pred == y_label).item() / len(y_label)  

score_f(tX_train,ty_train,nn1)
#Nos da un porcetaje de precision del 87% lo cual es muy positivo 

/tmp/ipykernel_2122/1969592921.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_label = torch.tensor(y, dtype=torch.long, requires_grad=False)   # recupera el valor histórico de y
/tmp/ipykernel_2122/2424509203.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_t = torch.tensor(x, dtype=torch.float, requires_grad=False)


0.844106463878327

In [54]:
#Probamos la validacion para asegurarnos que tambien sea estable 
score_f(tX_test,ty_test,nn1)
#Una precision muy similar y tambien elevada, es verdad que no se obtienen los mismos resulados entre el test y el entrenamiento 

/tmp/ipykernel_2122/1969592921.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_label = torch.tensor(y, dtype=torch.long, requires_grad=False)   # recupera el valor histórico de y
/tmp/ipykernel_2122/2424509203.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_t = torch.tensor(x, dtype=torch.float, requires_grad=False)


0.8142857142857143

In [55]:
Results = pd.DataFrame({
    'model':['Original'],
    'train': [score_f(tX_train,ty_train,nn1)],
    'test': [score_f(tX_test,ty_test,nn1)]
})
Results.set_index('model',inplace=True)

/tmp/ipykernel_2122/1969592921.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_label = torch.tensor(y, dtype=torch.long, requires_grad=False)   # recupera el valor histórico de y
/tmp/ipykernel_2122/2424509203.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_t = torch.tensor(x, dtype=torch.float, requires_grad=False)


In [56]:
#Podriamos itentar continuar mejorando el modelo a traves de la contruccion de una red mas compleja, con otra capa intermedia y esta vez usando 200 unidades, y luego otra de 100.
nn2 = torch.nn.Sequential(
    torch.nn.Linear(tX_train.shape[1], 200), 
    torch.nn.ReLU(), 
    torch.nn.Linear(200, 100), 
    torch.nn.ReLU(), 
    torch.nn.Linear(100, len(target_label.classes_)), 
    torch.nn.LogSoftmax(dim=1) # El output es la probabilidad de pertenencia a cada categoría
)

# reinicializamos los parámetros
do_seeds(12)
for param in nn2.parameters():
    torch.nn.init.uniform_(param, 0, 1) 

#En este caso aparte de estimar un learning rate para el modelo Gradiente Escolastico Descendiente, se le añade el regularizador Weight_decay, el cual castiga la perdida
opt2 = torch.optim.SGD(nn2.parameters(), lr=0.02, weight_decay=0.02)

for i in range(10000):
    opt2.zero_grad() # reseteamos los valores acumulados de las derivadas
    pred_y = nn2(tX_train) # calculamos nuestras predicciones con los parámetros actuales
    loss = torch.nn.functional.nll_loss(pred_y, ty_train) # calculamos la 'loss' con los resultados 
    if i%1000 == 0:
        print("Época: {}, Péŕdida: {:.4f}".format(i, loss.item()))
    loss.backward() # propagamos las derivadas de la 'loss' con respecto a nuestros parámetros
    opt2.step() # ejecutamos un paso de nuestro optimizador

Época: 0, Péŕdida: 53.4568


Época: 1000, Péŕdida: 0.2753
Época: 2000, Péŕdida: 0.1212
Época: 3000, Péŕdida: 0.1145
Época: 4000, Péŕdida: 0.1185
Época: 5000, Péŕdida: 0.1218
Época: 6000, Péŕdida: 0.1257
Época: 7000, Péŕdida: 0.1286
Época: 8000, Péŕdida: 0.1305
Época: 9000, Péŕdida: 0.1307


In [57]:
for param in nn2.parameters():
    param.requires_grad = False
print(score_f(tX_train, ty_train, nn2))

print(score_f(tX_test, ty_test, nn2))
#Lo cual lo hace incluso mas preciso

0.9581749049429658
0.9285714285714286


/tmp/ipykernel_2122/1969592921.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_label = torch.tensor(y, dtype=torch.long, requires_grad=False)   # recupera el valor histórico de y
/tmp/ipykernel_2122/2424509203.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_t = torch.tensor(x, dtype=torch.float, requires_grad=False)


In [60]:
Modelo_complejo = {'model':['Modelo_Complejo'],'train':[score_f(tX_train, ty_train, nn2)],'test':[score_f(tX_test, ty_test, nn2)]}
Results.loc['Modelo_complejo'] = Modelo_complejo

/tmp/ipykernel_2122/1969592921.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_label = torch.tensor(y, dtype=torch.long, requires_grad=False)   # recupera el valor histórico de y
/tmp/ipykernel_2122/2424509203.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_t = torch.tensor(x, dtype=torch.float, requires_grad=False)


In [62]:
Modelo_complejo = {'model':['Modelo_Complejo'],'train':score_f(tX_train, ty_train, nn2),'test':score_f(tX_test, ty_test, nn2)}
Results.loc['Modelo_complejo'] = Modelo_complejo

/tmp/ipykernel_2122/1969592921.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_label = torch.tensor(y, dtype=torch.long, requires_grad=False)   # recupera el valor histórico de y
/tmp/ipykernel_2122/2424509203.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_t = torch.tensor(x, dtype=torch.float, requires_grad=False)


In [63]:
Results

,train,test
model,,
Original,0.844106,0.814286
Modelo_complejo,0.958175,0.928571
